In [31]:
from dotenv import load_dotenv

load_dotenv()

True

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

### LangGraph

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages  # 매 단계마다 새로운 메시지를 추가함
from langchain_core.messages import AnyMessage # HumanMessage, AIMessage, SystemMessage, ToolMessage, FunctionMessage를 나타내는 추상화된 타입

# 상태 객체 생성용 클래스
# 딕셔너리 구조
class AgentState(TypedDict):
    messages: Annotated[AnyMessage, add_messages] # add_messages 리듀서 함수는 상태 키의 값(메시지)을 업데이트(덮어쓰는 대신 리스트에 추가)
    # messages: Annotated[list[str], add_messages] # add_messages 리듀서 함수는 상태 키의 값(메시지)을 업데이트(덮어쓰는 대신 리스트에 추가)

In [34]:
from langgraph.graph import StateGraph # 상태 기반의 그래프를 생성하는 클래스(에이전트의 상태 관리, Node와 Edge 연결)

# 그래프 객체
graph_builder = StateGraph(AgentState)

In [35]:
# 함수 노드
def generate(state: AgentState) -> AgentState:
    return {'messages': [llm.invoke(state["messages"])]}

In [36]:
# 그래프 객체에 노드 추가
graph_builder.add_node('generate', generate)

In [37]:
from langgraph.graph import START, END # 그래프의 시작점과 끝점을 명시적으로 정의하는 데 사용되는 특별한 상수

# 엣지로 노드 연결
graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

### 체크포인트
에이전트의 현재 상태를 저장, 중단된 시점 부터 다시 시작하거나, 특정 시점으로 돌아갈 수 있게 함

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
config = {"configurable": {"thread_id": "kim1"}}

In [49]:
# graph = graph_builder.compile()
graph = graph_builder.compile(checkpointer=memory) # 그래프를 실행 가능한 상태로 변환하는 메서드

In [50]:
# uv add grandalf
graph.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [51]:
from langchain_core.messages import HumanMessage

initial_state = {'messages': [HumanMessage("내 이름은 김일남이야")]}
graph.invoke(initial_state, config)
# graph.invoke(initial_state)

{'messages': [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}, id='c7e03135-6255-466e-b159-a8de0b560ce4'),
  AIMessage(content='네, 안녕하세요, 김일남 님! 만나서 반갑습니다. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7f7d789a-ba59-4ddb-8acb-e6a354c2b5ca-0', usage_metadata={'input_tokens': 8, 'output_tokens': 1106, 'total_tokens': 1114, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1091}})]}

In [52]:
initial_state = {'messages': [HumanMessage("내 이름은?")]}
graph.invoke(initial_state, config)
# graph.invoke(initial_state)

{'messages': [HumanMessage(content='내 이름은 김일남이야', additional_kwargs={}, response_metadata={}, id='c7e03135-6255-466e-b159-a8de0b560ce4'),
  AIMessage(content='네, 안녕하세요, 김일남 님! 만나서 반갑습니다. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7f7d789a-ba59-4ddb-8acb-e6a354c2b5ca-0', usage_metadata={'input_tokens': 8, 'output_tokens': 1106, 'total_tokens': 1114, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1091}}),
  HumanMessage(content='내 이름은?', additional_kwargs={}, response_metadata={}, id='352c4891-89ae-4994-9596-71782b571166'),
  AIMessage(content='김일남 님이십니다! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6e39145c-e74f-46b6-9108-6df6d3e66800-0', usage_metadat

In [43]:
while True:
    user_input = input("You: ")

    if user_input == "종료":
        break

    for event in graph.stream({'messages': [HumanMessage(user_input)]}, config=config):
        print(event["generate"]["messages"][-1].content)